<a href="https://colab.research.google.com/github/skyjadel/chang-s_titanic/blob/main/Titanic_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [2]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


先移除 Name，Ticket，Cabin<br>
Name 比較難以量化，可能可以找關聯性，但較為複雜，先剔除，同理移除ticket<br>
Cabin 缺失值太多，移除


In [3]:
df = df[list(set(df.columns).difference(['Ticket','Name','Cabin']))]
df

,Parch,PassengerId,Age,Pclass,Embarked,Survived,Sex,Fare,SibSp
0,0,1,22.0,3,S,0,male,7.2500,1
1,0,2,38.0,1,C,1,female,71.2833,1
2,0,3,26.0,3,S,1,female,7.9250,0
3,0,4,35.0,1,S,1,female,53.1000,1
4,0,5,35.0,3,S,0,male,8.0500,0
...,...,...,...,...,...,...,...,...,...
886,0,887,27.0,2,S,0,male,13.0000,0
887,0,888,19.0,1,S,1,female,30.0000,0
888,2,889,NaN,3,S,0,female,23.4500,1
889,0,890,26.0,1,C,1,male,30.0000,0


# 資料前處理

## 區分 Training Set 與 Testing Set

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=109)
print(f'{len(train_df)} rows in training set.')
print(f'{len(test_df)} rows in testing set.')

712 rows in training set.
179 rows in testing set.


## 補缺失值

從上次的EDA我們知道，有缺失值的欄位有 Age 和 Embarked，Age將補上training set 的平均值，Embarked將補上trainin set 的最多分類。


---

20240510 12:40 <br>
因為發現不同艙等的年齡分布有差別，而三等艙的旅客年齡缺失的機率又比較大，所以改填各艙等的平均年齡。

In [5]:
# 找出哪個港口上船的人最多
grouped = train_df.groupby('Embarked')
harbor_counts = {g[0]:len(g[1]) for g in grouped}

max_count = 0
for k, v in harbor_counts.items():
  if v > max_count:
    max_count = v
    major_embarked_harbor = k

In [6]:
# Training Set 的年齡平均值
#mean_age = train_df['Age'].mean()
mean_age = {i: train_df[train_df['Pclass']==i]['Age'].mean() for i in range(1,4)}

In [7]:
def fill_nan(df, age_value=mean_age, embarked_value=major_embarked_harbor):
  df = df.fillna({'Embarked':embarked_value})


  if type(age_value) == dict:
    age_is_na = df['Age'].isna()
    for i in df.index:
      if age_is_na[i]:
        df.loc[i, 'Age'] = mean_age[df.loc[i, 'Pclass']]
  else:
    df = df.fillna({'Age':age_value})

  return df

## Labeling

Sex 欄位改稱 Sex_female 欄位 <br>
Embarked 使用onehot <br>
註：這邊曾經嘗試不要刪除'Sex_male'，發現沒有任何改善，因次不採用這個更動

In [8]:
def labeling(df, remove_sex_male=True):
  df = pd.concat([df, pd.get_dummies(df[['Sex', 'Embarked']])], axis=1)
  if remove_sex_male:
    df.drop(['Sex', 'Embarked', 'Sex_male'], axis=1, inplace=True)
  else:
    df.drop(['Sex', 'Embarked'], axis=1, inplace=True)
  return df

## 添加特徵

前面發現單人登船者生還率較低，因此嘗試加入是否為單人特徵
(測試結果發現沒有比較好，停用)

In [9]:
def single(df):
  df['Single'] = 0
  for i in df.index:
    if sum(df.loc[i][['SibSp', 'Parch']])==0:
      df.loc[i, 'Single'] = 1
  return df

前面發現15歲以下生還率較高，因此嘗試加入是否為15歲以下特徵

In [10]:
def young(df):
  df['Young'] = 0
  for i in df.index:
    if df.loc[i]['Age']<=15:
      df.loc[i, 'Young'] = 1
  return df

前面發現60歲以下生還率較低，嘗試加入是否為60歲以下特徵(測試結果發現沒有比較好，停用)

In [11]:
def old(df):
  df['Old'] = 0
  for i in df.index:
    if df.loc[i]['Age']>60:
      df.loc[i, 'Old'] = 1
  return df

前面發現在票價低於100元時，票價越高生還率越高，加入票價binning(測試結果發現沒有比較好，停用)

In [12]:
def fare_binning(df, cuts=[25, 50, 75, 100]):
  df['Fare_Bin'] = 0
  for i in df.index:
    this_fb = 0
    this_fare = df.loc[i,'Fare']
    for c in cuts:
      if this_fare > c:
        this_fb += 1
      else:
        df.loc[i, 'Fare_Bin'] = this_fb
        break
  return df

## 前處理函式

In [13]:
def preprocess(df):
  df = labeling(fill_nan(df))
  #df = single(df)
  df = young(df)
  #df = old(df)
  #df = fare_binning(df)
  return df

In [14]:
train_df = preprocess(train_df)
test_df = preprocess(test_df)

In [15]:
train_df

,Parch,PassengerId,Age,Pclass,Survived,Fare,SibSp,Sex_female,Embarked_C,Embarked_Q,Embarked_S,Young
70,0,71,32.000000,2,0,10.5000,0,False,False,False,True,0
183,1,184,1.000000,2,1,39.0000,2,False,False,False,True,1
599,0,600,49.000000,1,1,56.9292,1,False,True,False,False,0
815,0,816,38.637973,1,0,0.0000,0,False,False,False,True,0
674,0,675,29.525926,2,0,0.0000,0,False,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,400,28.000000,2,1,12.6500,0,True,False,False,True,0
141,0,142,22.000000,3,1,7.7500,0,True,False,False,True,0
757,0,758,18.000000,2,0,11.5000,0,False,False,False,True,0
245,0,246,44.000000,1,0,90.0000,2,False,False,True,False,0


# 訓練

In [16]:
columns_X = list(set(train_df.columns) - {'Survived', 'PassengerId'})
columns_y = ['Survived']

train_X = train_df[columns_X]
train_y = train_df[columns_y]
test_X = test_df[columns_X]
test_y = test_df[columns_y]

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)
print(f'平均準確率{np.mean(scores):.4f}')

[0.74125874 0.84615385 0.84507042 0.72535211 0.87323944]
平均準確率0.8062


In [18]:
H = log.fit(np.array(train_X), np.array(train_y).reshape(-1))

In [19]:
pred_y = log.predict(np.array(train_X))
true_y = yt = np.array(train_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for training set is {acc_samples/len(train_df):.4f} ({acc_samples}/{len(train_df)})')

Accuracy for training set is 0.8118 (578/712)


In [20]:
pred_y = log.predict(np.array(test_X))
true_y = yt = np.array(test_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for testing set is {acc_samples/len(test_df):.4f} ({acc_samples}/{len(test_df)})')

Accuracy for testing set is 0.8156 (146/179)


# 結論

進行特徵工程之前的 accuracy 為  
cross validation: 0.7936, train-test 8:2 split: 0.7989  
目前嘗試過的特徵工程：
1. 年齡缺失值改填艙等平均年齡，acc: 0.8034, 0.8101 (採納)  
2. 加入是否為單身的特徵，acc:0.8076, 0.7989 (不採納)  
3. 加入是否小於15歲的特徵，acc: 0.8062, 0.8156 (採納)(目前最佳)  
4. 性別使用onehot encoding而非labelencoding，acc:  0.8034, 0.8156 (不採納)  
5. 加入是否大於60歲的特徵，acc: 0.8062, 0.8156 (不採納)  
6. 加入票價裝箱，acc: 0.8048, 0.8101 (不採納)